In [8]:
## STANDARD PYTHON LIBS
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## ADDING TO ROOT
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(ROOT_DIR)

## CUSTOM LIBS
import src.utils as utils
import src.model as modellib
from src.config import Config
import src.visualize as visualize

# GPU LIBS
import keras
import tensorflow as tf

cuda_version = os.popen("cat /usr/local/cuda/version.txt ").read()
print ('TF : ', tf.__version__, '  Keras : ', keras.__version__, '  CUDA : ', cuda_version)

TF :  1.6.0   Keras :  2.1.5   CUDA :  CUDA Version 9.0.176



In [9]:
if 'src.mapillary' in sys.modules : del sys.modules['src.mapillary']
if 'src.utils'     in sys.modules : del sys.modules['src.utils']
if 'src.model'     in sys.modules : del sys.modules['src.model']

import src.utils as utils
import src.model as modellib
import src.mapillary as mapillary

if __name__ == "__main__":
    url_dataset = '/home/play/datasets/open_datasets/mapillary'
    mapillary_mapper = '/home/play/playment/Mask_RCNN/demo/raw/merge__cityscapes_mapillary_v2.json'
    mapillary_config = mapillary.MapillaryConfig()
    mapillary_config.IMAGES_PER_GPU = 8
    mapillary_config.STEPS_PER_EPOCH = 10
    trainData = mapillary.MapillaryDataset(url_dataset, mapillary_mapper, mapillary_config, data_type = 'train')
    valData   = mapillary.MapillaryDataset(url_dataset, mapillary_mapper, mapillary_config, data_type = 'val')

Mode : train has 17979 images
Mode : val has 2000 images


In [ ]:
import h5py
import time

def create_hdf5(generator, url_dataset, counter):
    print ('Saving in dir : ', url_dataset)
    iters = 0
    while True:
        t0 = time.time()
        url_tmp = os.path.join(url_dataset, '{}.h5'.format(iters))
        data = next(generator)
        if len(data) == 2:
            ip = data[0]
            op = data[1]
        with h5py.File(url_tmp, "w") as hf:
            for i, each in enumerate(ip):
                hf.create_dataset('input_{0}'.format(i), data=each)
            if len(op):
                for i, each in enumerate(op):
                    hf.create_dataset('output_{0}'.format(i), data=each)
            else:
                hf.create_dataset('output'.format(i), data=op)

        print (' -------> Time taken for save : {0} || URL : {1}'.format(round(time.time() - t0,2), iters))
        iters += 1
        if iters > counter:
            break

print ('mapillary_config.BATCH_SIZE : ', mapillary_config.BATCH_SIZE)
train_generator = modellib.data_generator(trainData, mapillary_config, shuffle=True, batch_size=mapillary_config.BATCH_SIZE)
url_dataset_train = '/home/play/datasets/open_datasets/mapillary_hdf5/mapillary-vistas-dataset_public_v1.0/training'
create_hdf5(train_generator, url_dataset_train, counter = 2250)

mapillary_config.BATCH_SIZE :  8
Saving in dir :  /home/play/datasets/open_datasets/mapillary_hdf5/mapillary-vistas-dataset_public_v1.0/training
 -------> Time taken for save : 46.44 || URL : 0
 ----> Time Taken for one batch 46.43 || Final Index : 7
 -------> Time taken for save : 48.22 || URL : 1
 ----> Time Taken for one batch 48.22 || Final Index : 15
 -------> Time taken for save : 38.54 || URL : 2
 ----> Time Taken for one batch 38.54 || Final Index : 23
 -------> Time taken for save : 39.91 || URL : 3
 ----> Time Taken for one batch 39.91 || Final Index : 31
 -------> Time taken for save : 47.53 || URL : 4
 ----> Time Taken for one batch 47.53 || Final Index : 39
 -------> Time taken for save : 47.96 || URL : 5
 ----> Time Taken for one batch 47.96 || Final Index : 47
 -------> Time taken for save : 37.36 || URL : 6
 ----> Time Taken for one batch 37.36 || Final Index : 55
 -------> Time taken for save : 46.89 || URL : 7
 ----> Time Taken for one batch 46.89 || Final Index : 63


 -------> Time taken for save : 74.36 || URL : 75
 ----> Time Taken for one batch 74.36 || Final Index : 607
 -------> Time taken for save : 56.4 || URL : 76
 ----> Time Taken for one batch 56.4 || Final Index : 615
 -------> Time taken for save : 61.02 || URL : 77
 ----> Time Taken for one batch 61.02 || Final Index : 623
 -------> Time taken for save : 39.22 || URL : 78
 ----> Time Taken for one batch 39.22 || Final Index : 631
 -------> Time taken for save : 48.85 || URL : 79
 ----> Time Taken for one batch 48.85 || Final Index : 639
 -------> Time taken for save : 39.65 || URL : 80
 ----> Time Taken for one batch 39.65 || Final Index : 647
 -------> Time taken for save : 46.67 || URL : 81
 ----> Time Taken for one batch 46.67 || Final Index : 655
 -------> Time taken for save : 47.28 || URL : 82
 ----> Time Taken for one batch 47.28 || Final Index : 663
 -------> Time taken for save : 58.02 || URL : 83
 ----> Time Taken for one batch 58.02 || Final Index : 671
 -------> Time taken 

# GENERATOR

In [5]:
def data_generator(url_dataset):
    while True:
        for each in os.listdir(dataset):
            url_tmp = os.path.join(url_dataset_train, each)
            ip, op = [], []
            print (url_tmp)
            with h5py.File(url_tmp, "r") as hf:
                ip, op = [], []
                for name in hf:
                    if 'input' in name:
                        ip.append(np.array(hf.get(name)))
                    elif 'output' in name:
                        op.append(np.array(hf.get(name)))
            yield ip, op[0]

gen = data_generator('', '')

In [6]:
next(gen)

NameError: name 'url_dataset_train' is not defined